<a href="https://colab.research.google.com/github/hiromasat/ML/blob/master/Tabular_Playground_Series_May_2021.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# kaggleモジュールをインポート
!pip install kaggle

from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"hiromasatabuchi","key":"e3c87bc19bde10d0845a4c449fc87abb"}'}

# 新しいセクション

In [ ]:
import os
os.makedirs('/root/.kaggle', exist_ok=True)

import shutil
# アップロードしたファイルを移動する方法
shutil.copyfile('/content/kaggle.json', '/root/.kaggle/kaggle.json')

# Google Drive上からコピーする方法
# shutil.copyfile('/content/drive/MyDrive/path/to/kaggle.json', '/root/.kaggle/kaggle.json')

'/root/.kaggle/kaggle.json'

In [ ]:
!kaggle competitions download -c tabular-playground-series-may-2021
!unzip '/content/train.csv.zip' -d '/content/'
!unzip '/content/test.csv.zip' -d '/content/'

  0% 0.00/1.72M [00:00<?, ?B/s]
100% 1.72M/1.72M [00:00<00:00, 57.9MB/s]
  0% 0.00/128k [00:00<?, ?B/s]
100% 128k/128k [00:00<00:00, 126MB/s]
  0% 0.00/851k [00:00<?, ?B/s]
100% 851k/851k [00:00<00:00, 117MB/s]
Archive:  /content/train.csv.zip
  inflating: /content/train.csv      
Archive:  /content/test.csv.zip
  inflating: /content/test.csv       


In [ ]:
# file path(train data and test data)
train_file = '/content/train.csv'
test_file = '/content/test.csv'

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import time

from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

from sklearn.impute import SimpleImputer

# Dimension compression
from sklearn.feature_selection import RFE

# Algorithm
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
import xgboost as xgb
import lightgbm as lgb

# widget
from ipywidgets import interact,interactive,fixed,interact_manual
from IPython.display import display
import ipywidgets as widgets

In [ ]:
# column name for submission file
ID_column = 'id'

# column name of the target data
target = 'target'
submission_cols = ['Class_1', 'Class_2', 'Class_3', 'Class4']

In [ ]:
unuse_columns_for_train = [ID_column,]
train_reject_items = unuse_columns_for_train + [target]

In [ ]:
# Change view options
pd.options.display.max_columns = 50

In [ ]:
# read csv file
dataset = pd.read_csv(train_file, header=0)

# To drop reject items columns from train data set
# pandas.core.frame.DataFrame
X = pd.DataFrame(dataset).drop(columns=train_reject_items, axis=1)

# pandas.core.series.Series
y = pd.Series(dataset[target])

# check the shape
print('----------------------------------------------------------------')
print('X shape: (%i,%i)' %X.shape)
print('----------------------------------------------------------------')
print('y shape: (%i,)' %y.shape)
print('----------------------------------------------------------------')
print(y.value_counts())
print('----------------------------------------------------------------')
print()
print('dataset:csv file raw data')
display(dataset.head())
print('X: Train dataset')
X.join(y).head()

----------------------------------------------------------------
X shape: (100000,50)
----------------------------------------------------------------
y shape: (100000,)
----------------------------------------------------------------
Class_2    57497
Class_3    21420
Class_4    12593
Class_1     8490
Name: target, dtype: int64
----------------------------------------------------------------

dataset:csv file raw data


,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,...,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,3,0,0,21,0,0,0,0,0,0,Class_2
1,1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
2,2,0,0,0,0,0,0,0,0,0,2,0,0,1,0,1,0,0,0,0,1,0,5,2,1,...,1,0,2,0,0,0,0,19,2,0,1,7,3,0,0,0,1,0,0,0,0,13,2,0,Class_1
3,3,0,0,0,0,0,0,0,3,0,0,0,0,0,1,7,11,1,0,3,1,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,Class_4
4,4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,9,3,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,Class_2


X: Train dataset


,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,feature_24,...,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,1,0,0,0,0,0,3,0,0,21,0,0,0,0,0,0,Class_2
1,0,0,0,0,2,1,0,0,0,0,0,0,0,0,0,3,0,3,0,0,0,0,0,0,0,...,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Class_1
2,0,0,0,0,0,0,0,0,0,2,0,0,1,0,1,0,0,0,0,1,0,5,2,1,1,...,1,0,2,0,0,0,0,19,2,0,1,7,3,0,0,0,1,0,0,0,0,13,2,0,Class_1
3,0,0,0,0,0,0,0,3,0,0,0,0,0,1,7,11,1,0,3,1,0,0,0,0,0,...,0,0,1,0,1,0,0,0,1,0,0,0,4,0,0,0,0,0,0,0,0,0,1,0,Class_4
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,9,3,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,Class_2


In [ ]:
# Check missing value in each column of dataset
dataset.isnull().sum()

id            0
feature_0     0
feature_1     0
feature_2     0
feature_3     0
feature_4     0
feature_5     0
feature_6     0
feature_7     0
feature_8     0
feature_9     0
feature_10    0
feature_11    0
feature_12    0
feature_13    0
feature_14    0
feature_15    0
feature_16    0
feature_17    0
feature_18    0
feature_19    0
feature_20    0
feature_21    0
feature_22    0
feature_23    0
feature_24    0
feature_25    0
feature_26    0
feature_27    0
feature_28    0
feature_29    0
feature_30    0
feature_31    0
feature_32    0
feature_33    0
feature_34    0
feature_35    0
feature_36    0
feature_37    0
feature_38    0
feature_39    0
feature_40    0
feature_41    0
feature_42    0
feature_43    0
feature_44    0
feature_45    0
feature_46    0
feature_47    0
feature_48    0
feature_49    0
target        0
dtype: int64

In [ ]:
dic={"Class_1": 0, "Class_2":1, "Class_3": 2, "Class_4": 3}
y.replace(dic, inplace=True)

In [ ]:
X_columns =  X.columns.values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=46, stratify=y)

In [ ]:
# set pipelines for different algorithms
pipelines = {
    #'knn':
    #   Pipeline([('scl',StandardScaler()),
    #                ('est',KNeighborsClassifier())]),
    #'logistic':
    #    Pipeline([('scl',StandardScaler()),
    #                ('est',LogisticRegression(random_state=1))]),
    #'rsvc':
    #   Pipeline([('scl',StandardScaler()),
    #                ('est',SVC(C=1.0, kernel='rbf', class_weight='balanced', random_state=1))]),
    #'lsvc':
    #    Pipeline([('scl',StandardScaler()),
    #                ('est',LinearSVC(C=1.0, class_weight='balanced',random_state=1))]),
    #'tree':
    #    Pipeline([('scl',StandardScaler()),
    #                ('est',DecisionTreeClassifier(random_state=1))]),
    #'rf':
    #   Pipeline([('scl',StandardScaler()),
    #                ('est',RandomForestClassifier(random_state=1))]),
    #'gb':
    #    Pipeline([('scl',StandardScaler()),
    #                ('est',GradientBoostingClassifier(random_state=1))]),
    #'mlp':
    #    Pipeline([('scl',StandardScaler()),
    #                ('est',MLPClassifier(hidden_layer_sizes=(3,3), max_iter=1000, random_state=1))]),
    #'xgb':
    #    Pipeline([('scl',StandardScaler()),
    #                ('est',xgb.XGBClassifier())]),
    'lgbm':
        Pipeline([('scl',StandardScaler()),
                    ('est',lgb.LGBMClassifier())]),
}

In [ ]:
params = {
    #'knn' : {'est__n_neighbors':[5, 7,10],
    #         'est__weights':['uniform','distance'],},
    
    #'logistic': {'est__C':[1, 100],},
    
    #'rsvc': {'est__C':[1, 100],},
    
    #'lsvc': {'est__C':[1, 100],},
    
    #'tree': {'est__max_depth': list(range(10, 20)),
    #        'est__criterion': ['gini', 'entropy'],},
    
    #'rf': {'est__n_estimators':[100, 300],
    #        'est__max_depth': [8, 10,],
    #        'est__random_state': [0],},
    
    #'gb': {'est__loss':['deviance'],
    #        'est__learning_rate': [0.01, 0.1],
    #        'est__min_samples_split': np.linspace(0.1, 0.5, 2),
    #        'est__min_samples_leaf': np.linspace(0.1, 0.5, 2),
    #        'est__max_depth':[3,5],
    #        'est__max_features':['log2','sqrt'],
    #        'est__criterion': ['friedman_mse',  'mae'],
    #        'est__subsample':[0.5, 1.0],
    #        'est__n_estimators':[10],},
    
    #'mlp': {'est__solver': ['lbfgs'],
    #        'est__max_iter': [10000],
    #        'est__alpha': 10.0 ** -np.arange(1, 3),
    #        'est__hidden_layer_sizes':np.arange(10, 12),},
    
    # 'xgb': {'est__n_estimators':[100, ],
    #        'est__max_depth':[6,],
    #        'est__learning_rate':[0.01, ],
    #        'est__min_child_weight': [6, ],},
    
    'lgbm': {'est__max_depth':[30 ,50, 70,],
            'est__learning_rate':[0.01, 0.05, 0.1,],
            'est__num_leaves':[15, 31, 64,],
            'est__n_estimators':[50, 100, 200,],},
}

In [ ]:
lgm_params= {'objective': 'binary',
             'learning_rate': 0.01,
             'feature_pre_filter': False,
             'lambda_l1': 0.00019887452062866844,
             'lambda_l2': 1.6090525653083073e-07,
             'num_leaves': 194,
             'feature_fraction': 0.4,
             'bagging_fraction': 1.0,
             'bagging_freq': 0,
             'min_child_samples': 20,
             'num_iterations': 10000,
             'early_stopping_round': 10}

In [ ]:
#  評価指標
evaluation_scoring = 'accuracy'

# スコア格納用リストの初期化（Dict型）
scores = {}

# GridSearchのパラメータ、スコア格納用リストの初期化
best_params ={}
best_scores ={}

# パイプラインの先頭にある文字列（例えば、'KNN')が pipe_name に、各パイプラインのインスタンスがpipelineに順次入る
for pipe_name, pipeline in pipelines.items():
    print(pipe_name)
    print(params[pipe_name])
    start = time.time()
    gs = GridSearchCV(estimator=pipeline,
                     param_grid = params[pipe_name],
                     scoring=evaluation_scoring,
                     cv=5,
                     return_train_score=False)
    # 学習
    gs.fit(X_train, y_train)
    
    print('time', time.time()-start)
    scores[(pipe_name,'train')] = accuracy_score(y_train, gs.predict(X_train))
    scores[(pipe_name,'test')] = accuracy_score(y_test, gs.predict(X_test))
    best_params[pipe_name] = gs.best_params_
    best_scores[pipe_name] = gs.best_score_
    
    # 各モデル格納用のディレクトリ を作成
    os.makedirs('/kaggle/working/models/pipeline_models', exist_ok=True)
     # 各モデル保存(modelフォルダー)
    file_name = '/kaggle/working/models/pipeline_models/'+pipe_name+str(time.time())+'.pkl'
    pickle.dump(pipeline, open(file_name, 'wb'))

print('---accuracy---')
pd.Series(scores).unstack()

lgbm
{'est__objective': 'binary', 'est__learning_rate': 0.01, 'est__feature_pre_filter': False, 'est__lambda_l1': 0.00019887452062866844, 'est__lambda_l2': 1.6090525653083073e-07, 'est__num_leaves': 194, 'est__feature_fraction': 0.4, 'est__bagging_fraction': 1.0, 'est__bagging_freq': 0, 'est__min_child_samples': 20, 'est__num_iterations': 10000, 'est__early_stopping_round': 10}


ValueError: ignored

In [ ]:
def get_answer(x):
    return x

In [ ]:
model_selection = get_answer(widgets.RadioButtons(options=pipelines.keys()))
display(model_selection)

RadioButtons(options=('lgbm',), value='lgbm')

In [ ]:
selected_model_name = model_selection.value
print(selected_model_name)

lgbm


In [ ]:
best_params[selected_model_name]

{'est__learning_rate': 0.1,
 'est__max_depth': 50,
 'est__n_estimators': 100,
 'est__num_leaves': 64}

In [ ]:
def train_on_selected_model(pipe_name, pipelines, params, evaluation_scoring, tag):
    start = time.time()
    gs = GridSearchCV(estimator=pipelines[pipe_name],
                     param_grid = params,
                     scoring=evaluation_scoring,
                     cv=5,
                     return_train_score=False)
    # 学習
    # model = gs.fit(X_train, y_train)
    model = gs.fit(X, y) 
   
    print('time', time.time()-start)
    scores[(pipe_name,'train')] = accuracy_score(y_train, gs.predict(X_train))
    scores[(pipe_name,'test')] = accuracy_score(y_test, gs.predict(X_test))
    
    # 各モデル格納用のディレクトリ を作成
    os.makedirs('/kaggle/working/models/', exist_ok=True)
     # 各モデル保存(modelフォルダー)
    file_name = '/kaggle/working/models/' + pipe_name + '_' + tag + str(time.time())+'.pkl'
    pickle.dump(model, open(file_name, 'wb'))
    
    return model

In [ ]:
# best_params[seleced_model_name]のvalueをリスト化する
def params_parser(best_params, selected_model_name):
    return {k:[v] for k,v in best_params[selected_model_name].items()}

best_param_parsed = params_parser(best_params, selected_model_name)
print(best_param_parsed)

{'est__learning_rate': [0.1], 'est__max_depth': [50], 'est__n_estimators': [100], 'est__num_leaves': [64]}


In [ ]:
selected_model = train_on_selected_model(selected_model_name, pipelines, best_param_parsed, evaluation_scoring, 'selected')

time 66.42017793655396


In [ ]:
scores

{('lgbm', 'test'): 0.5958, ('lgbm', 'train'): 0.5941875}

In [ ]:
# csvファイルからの読み出し
dataset_s = pd.read_csv(test_file,header=0)

# 1列目のID情報、推論対象は特徴量から削除
ID_s = dataset_s.iloc[:,[0]]    # 第０列はPassengerIDなので、IDとしてセット
X_s = dataset_s.drop(unuse_columns_for_train, axis=1)    # 1列目はID情報のため特徴量から削除

# 形状の確認
print('-----------------------------------')
print('Raw Shape: (%i, %i)' %dataset_s.shape)
print('X_s Shape: (%i, %i)' %X_s.shape)
print('-----------------------------------')
print(X_s.dtypes)
ID_s.join(X_s).head()

-----------------------------------
Raw Shape: (50000, 51)
X_s Shape: (50000, 50)
-----------------------------------
feature_0     int64
feature_1     int64
feature_2     int64
feature_3     int64
feature_4     int64
feature_5     int64
feature_6     int64
feature_7     int64
feature_8     int64
feature_9     int64
feature_10    int64
feature_11    int64
feature_12    int64
feature_13    int64
feature_14    int64
feature_15    int64
feature_16    int64
feature_17    int64
feature_18    int64
feature_19    int64
feature_20    int64
feature_21    int64
feature_22    int64
feature_23    int64
feature_24    int64
feature_25    int64
feature_26    int64
feature_27    int64
feature_28    int64
feature_29    int64
feature_30    int64
feature_31    int64
feature_32    int64
feature_33    int64
feature_34    int64
feature_35    int64
feature_36    int64
feature_37    int64
feature_38    int64
feature_39    int64
feature_40    int64
feature_41    int64
feature_42    int64
feature_43    int64
fe

,id,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,feature_10,feature_11,feature_12,feature_13,feature_14,feature_15,feature_16,feature_17,feature_18,feature_19,feature_20,feature_21,feature_22,feature_23,...,feature_25,feature_26,feature_27,feature_28,feature_29,feature_30,feature_31,feature_32,feature_33,feature_34,feature_35,feature_36,feature_37,feature_38,feature_39,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,100000,0,0,0,0,0,0,4,4,0,0,5,0,0,0,0,1,1,0,1,0,1,0,0,0,...,0,5,0,5,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,0,0,0,0,0,0
1,100001,0,0,1,0,0,0,2,0,7,7,0,0,0,0,0,3,0,0,2,0,0,0,0,0,...,0,4,0,2,0,0,2,0,3,6,3,0,0,6,0,3,0,1,0,0,0,1,0,2,1
2,100002,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,20,0,0,0,0,...,4,0,0,0,0,0,3,0,0,0,0,0,2,11,0,0,0,0,1,0,0,0,0,6,0
3,100003,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,1,0,0,0,4,1,0,0,1,...,0,1,1,1,0,0,0,1,4,0,0,0,0,0,0,0,0,0,0,0,0,6,9,14,3
4,100004,0,0,0,0,0,0,1,0,4,0,0,1,0,0,9,0,0,0,0,0,0,3,0,0,...,0,0,0,2,0,0,1,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0


In [ ]:
X_columns =  X.columns.values
print(X_columns)

['feature_0' 'feature_1' 'feature_2' 'feature_3' 'feature_4' 'feature_5'
 'feature_6' 'feature_7' 'feature_8' 'feature_9' 'feature_10' 'feature_11'
 'feature_12' 'feature_13' 'feature_14' 'feature_15' 'feature_16'
 'feature_17' 'feature_18' 'feature_19' 'feature_20' 'feature_21'
 'feature_22' 'feature_23' 'feature_24' 'feature_25' 'feature_26'
 'feature_27' 'feature_28' 'feature_29' 'feature_30' 'feature_31'
 'feature_32' 'feature_33' 'feature_34' 'feature_35' 'feature_36'
 'feature_37' 'feature_38' 'feature_39' 'feature_40' 'feature_41'
 'feature_42' 'feature_43' 'feature_44' 'feature_45' 'feature_46'
 'feature_47' 'feature_48' 'feature_49']


In [ ]:
X_s = X_s.loc[:, X_columns]

print('-----------------------------------')
print('X_s shape: (%i,%i)' %X_s.shape)
print('-----------------------------------')

-----------------------------------
X_s shape: (50000,50)
-----------------------------------


In [ ]:
X_s.columns.values

array(['feature_0', 'feature_1', 'feature_2', 'feature_3', 'feature_4',
       'feature_5', 'feature_6', 'feature_7', 'feature_8', 'feature_9',
       'feature_10', 'feature_11', 'feature_12', 'feature_13',
       'feature_14', 'feature_15', 'feature_16', 'feature_17',
       'feature_18', 'feature_19', 'feature_20', 'feature_21',
       'feature_22', 'feature_23', 'feature_24', 'feature_25',
       'feature_26', 'feature_27', 'feature_28', 'feature_29',
       'feature_30', 'feature_31', 'feature_32', 'feature_33',
       'feature_34', 'feature_35', 'feature_36', 'feature_37',
       'feature_38', 'feature_39', 'feature_40', 'feature_41',
       'feature_42', 'feature_43', 'feature_44', 'feature_45',
       'feature_46', 'feature_47', 'feature_48', 'feature_49'],
      dtype=object)

In [ ]:
y_pred = selected_model.predict_proba(X_s)
print(len(y_pred))

50000


In [ ]:
y_pred

array([[0.07738215, 0.56721028, 0.23019082, 0.12521674],
       [0.05843835, 0.70161804, 0.15071391, 0.0892297 ],
       [0.08796346, 0.64147534, 0.18249615, 0.08806505],
       ...,
       [0.08332659, 0.54147304, 0.21899775, 0.15620262],
       [0.07998525, 0.62143165, 0.16057407, 0.13800903],
       [0.09483843, 0.59023457, 0.19140985, 0.12351715]])

In [ ]:
id_df=pd.DataFrame(data=ID_s,columns=[ID_column])

submission_column_list =  ['Class_1','Class_2', 'Class_3', 'Class_4']
result_df = pd.DataFrame(data=y_pred, columns=submission_column_list)

submit_df = pd.concat([id_df, result_df], axis=1)

submit_df.head()

,id,Class_1,Class_2,Class_3,Class_4
0,100000,0.077382,0.567210,0.230191,0.125217
1,100001,0.058438,0.701618,0.150714,0.089230
2,100002,0.087963,0.641475,0.182496,0.088065
3,100003,0.089276,0.576659,0.220700,0.113364
4,100004,0.072456,0.615538,0.211017,0.100990


In [ ]:
# Setting for submission file
submit_file_dir = '/content/submit/'
submit_file_name = 'submission.csv'

# make directories for submittion file
os.makedirs(submit_file_dir, exist_ok=True)

# To generate csv file for submittion
submit_df.to_csv(submit_file_dir + selected_model_name + '_' + submit_file_name , index=False)